In [1]:
from graphtoolbox import OgbDataLoader, GraphHelper
from ogb.graphproppred import GraphPropPredDataset
import numpy as np
from tqdm import tqdm

/home/cstaib/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset_name = 'ogbg-molhiv'

In [3]:
ogb_data_loader = OgbDataLoader()
X_raw_graphs, y, split_idx = ogb_data_loader.get_processed_dataset(dataset_name)

100%|██████████| 41127/41127 [00:10<00:00, 3775.21it/s]


In [ ]:
graph_helper = GraphHelper()

X_graphs = [graph_helper.edges_to_nodes(graph) for graph in tqdm(X_raw_graphs)]

In [5]:
from graphtoolbox import RandomWalkEmbedder

model = RandomWalkEmbedder(attributed=True)

documents = model.generate_random_walks(X_raw_graphs)

 39%|███▉      | 16136/41127 [00:55<01:18, 316.92it/s]

In [ ]:
text_file = open("documents.txt", "w")
n = text_file.write(documents)
text_file.close()

In [ ]:
# from karateclub import Graph2Vec
# 
# model = Graph2Vec(attributed=True)
# model.fit(X_graphs)
# X = model.get_embedding()

In [ ]:
from graphtoolbox import RandomWalkEmbedder

model = RandomWalkEmbedder(attributed=True)
model.fit(X_graphs)
X = model.get_embedding()

In [ ]:
X.shape

In [ ]:
y[0].shape

In [ ]:
X_train = np.array([X[i] for i in split_idx['train']])
y_train = np.array([y[i] for i in split_idx['train']]).ravel()

X_test = np.array([X[i] for i in split_idx['test']])
y_test = np.array([y[i] for i in split_idx['test']]).ravel()

In [1]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
clf.fit(X_train, y_train)
y_test_predictions = clf.predict(X_test)

NameError: name 'X_train' is not defined

In [ ]:
from ogb.graphproppred import Evaluator

evaluator = Evaluator(name = dataset_name)

input_dict = {"y_true": y_test.reshape(-1, 1), "y_pred": y_test_predictions.reshape(-1, 1)}
result_dict = evaluator.eval(input_dict)

In [ ]:
result_dict